# LTV by Risk Segment (Colab)
Authenticate and run the cells to query BigQuery and visualize LTV.

In [ ]:
!pip install --quiet google-cloud-bigquery pandas matplotlib seaborn
from google.colab import auth
auth.authenticate_user()
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
client=bigquery.Client()
query='''
WITH customer_ltv AS (
  SELECT t.customer_id, d.risk_segment, SUM(t.amount) AS lifetime_value
  FROM `scd2-ltv-analysis.tracking_risco_cliente.fact_transactions` t
  JOIN `scd2-ltv-analysis.tracking_risco_cliente.dim_customers` d
    ON t.customer_id = d.customer_id
   AND t.transaction_date BETWEEN d.valid_from AND COALESCE(d.valid_to, CURRENT_DATE())
  GROUP BY t.customer_id, d.risk_segment
)
SELECT risk_segment, COUNT(DISTINCT customer_id) AS num_customers, ROUND(AVG(lifetime_value),2) AS avg_ltv
FROM customer_ltv
GROUP BY risk_segment
ORDER BY avg_ltv DESC
'''
ltv_df = client.query(query).to_dataframe()
print(ltv_df)
plt.bar(ltv_df['risk_segment'], ltv_df['avg_ltv'])
plt.title('Average LTV by Risk Segment')
plt.show()
